In [1]:
!pip3 install playwright
!pip3 install selectolax

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 34.6 MB/s eta 0:00:00


In [2]:
!playwright install chromium

163.5 MiB [] 0% 124.4s163.5 MiB [] 0% 788.0s163.5 MiB [] 0% 435.5s163.5 MiB [] 0% 325.3s163.5 MiB [] 0% 171.6s163.5 MiB [] 0% 34.3s163.5 MiB [] 0% 18.8s163.5 MiB [] 0% 16.3s163.5 MiB [] 1% 12.6s163.5 MiB [] 1% 12.4s163.5 MiB [] 1% 12.2s163.5 MiB [] 1% 12.1s163.5 MiB [] 1% 11.7s163.5 MiB [] 2% 11.8s163.5 MiB [] 2% 12.2s163.5 MiB [] 2% 12.1s163.5 MiB [] 2% 12.0s163.5 MiB [] 3% 11.0s163.5 MiB [] 3% 10.4s163.5 MiB [] 3% 10.0s163.5 MiB [] 4% 9.2s163.5 MiB [] 4% 9.1s163.5 MiB [] 4% 9.3s163.5 MiB [] 4% 9.0s163.5 MiB [] 5% 9.0s163.5 MiB [] 5% 9.1s163.5 MiB [] 5% 8.7s163.5 MiB [] 5% 8.9s163.5 MiB [] 5% 9.1s163.5 MiB [] 5% 8.9s163.5 MiB [] 6% 8.5s163.5 MiB [] 6% 9.0s163.5 MiB [] 6% 9.4s163.5 MiB [] 6% 9.3s163.5 MiB [] 6% 9.2s163.5 MiB [] 6% 9.3s163.5 MiB [] 7% 8.9s163.5 MiB [] 7% 8.5s163.5 MiB [] 8% 8.5s163.5 MiB [] 8% 8.1s163.5 MiB [] 8% 8.2s163.5 MiB [] 8% 8.5s163.5 MiB [] 9% 8.2s163.5 MiB [] 9% 8.0s163.5 MiB [] 9% 8.1s163.5 MiB [] 9% 8.2s163.5 MiB [] 9% 8.3s163.5 MiB [] 9% 8.4s163.5 MiB [] 10

In [4]:
url = "https://store.steampowered.com/specials" # Javascript-heavy website

In [5]:
# import asyncio
# from playwright.async_api import async_playwright

# async with async_playwright() as p:
#     browser = await p.chromium.launch(headless=True)
#     page = await browser.new_page()
#     await page.goto(url)
#     # await page.screenshot(path="screenshot.png", full_page=True) # From screenshot, we can see not everything is loaded in the webpage so our parsing strategy will fail.

#     TIMEOUT = 300001
#     await page.wait_for_load_state('networkidle', timeout=TIMEOUT) # When there is nothing left to render and network is literally idle
#     await page.evaluate("() => window.scroll(0, document.body.scrollHeight)") # Sometimes, for performance, websites renders information as user scrolls down to the page. So, to get full page content, we have to imitate scroll down on the website
#     await page.screenshot(path="screenshot2.png", full_page=True) # From screenshot2, we can see everything is loaded now.
#     await browser.close()

In [25]:
import asyncio
from playwright.async_api import async_playwright

async def extract_full_body_html(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        await page.goto(url, wait_until="domcontentloaded")

        # Repeatedly scroll until we reach the bottom.
        previous_height = 0
        while True:
            # Scroll to the bottom
            await page.evaluate("window.scrollTo(0, document.body.scrollHeight)")

            # Wait briefly for any new content to load
            await page.wait_for_timeout(2000)

            # Check the new scroll height
            new_height = await page.evaluate("document.body.scrollHeight")
            if new_height == previous_height:
                # No new content was loaded, so we’re done
                break
            previous_height = new_height

        html = await page.inner_html("body")

        await asyncio.sleep(2)

        # Now that we've scrolled fully, take the screenshot
        await page.screenshot(path="full_screenshot.png", full_page=True)

        await browser.close()

        return html

def get_divs(html):
    tree = HTMLParser(html)
    divs = tree.css('div[class*="gASJ2lL_xmVNuZkWGvrWg"]') # This class name keeps changing. That is a problem
    return divs

# Running inside Jupyter Notebook
async def main():
    html = await extract_full_body_html(url)
    print(get_divs(html), len(get_divs(html)))

# Run the async function properly
await main()



[<Node div>, <Node div>, <Node div>, <Node div>, <Node div>, <Node div>, <Node div>, <Node div>, <Node div>, <Node div>, <Node div>, <Node div>] 12
